In [69]:
from __future__ import print_function, division
#virtualenv -p python3 venv

In [70]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [71]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [72]:
ls ../data/train/

Type_1/ Type_2/ Type_3/


In [73]:
df = c.data.make_base_df()
tdf = c.data.make_test_df(); tdf.head()

,path,num_id
0,../data/test/0.jpg,0
1,../data/test/1.jpg,1
2,../data/test/10.jpg,10
3,../data/test/100.jpg,100
4,../data/test/101.jpg,101


In [74]:
df = c.processing.transform_save_imgs(df, c.processing.resize_100); df.head()

,path,type,filetype,num_id,resize_100_path
0,../data/train/Type_1/0.jpg,Type_1,jpg,0,../data/processed/resize_100/train/0.jpg
1,../data/train/Type_1/10.jpg,Type_1,jpg,10,../data/processed/resize_100/train/10.jpg
2,../data/train/Type_1/1013.jpg,Type_1,jpg,1013,../data/processed/resize_100/train/1013.jpg
3,../data/train/Type_1/1014.jpg,Type_1,jpg,1014,../data/processed/resize_100/train/1014.jpg
4,../data/train/Type_1/1019.jpg,Type_1,jpg,1019,../data/processed/resize_100/train/1019.jpg


In [75]:
tdf = c.processing.transform_save_imgs(tdf, c.processing.resize_100); tdf.head()

,path,num_id,resize_100_path
0,../data/test/0.jpg,0,../data/processed/resize_100/test/0.jpg
1,../data/test/1.jpg,1,../data/processed/resize_100/test/1.jpg
2,../data/test/10.jpg,10,../data/processed/resize_100/test/10.jpg
3,../data/test/100.jpg,100,../data/processed/resize_100/test/100.jpg
4,../data/test/101.jpg,101,../data/processed/resize_100/test/101.jpg


In [78]:
X, y = c.processing.random_forest_transform(df, 'resize_100_path', grayscale=False)
print(X.shape)
print(y.shape)

(1481, 30000)
(1481,)


In [65]:
import scipy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [66]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(992, 10000)
(992,)
(489, 10000)
(489,)


In [67]:
from sklearn.ensemble import RandomForestClassifier as RFC
rfc = RFC(n_jobs=2,n_estimators=50)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [21]:
rfc.score(X_test, y_test)  

0.48466257668711654

In [28]:
df = c.processing.transform_save_imgs(df, c.processing.resize_100); df.head()

1131.1134.1136.1139.1154.1161.1168.1174.1179.1190.1194.1199.12.1202.1204.1220.1223.1226.1229.1230.1239.1245.1251.1273.1274.1279.1281.1285.1288.1289.129.13.1308.1314.1320.1324.1326.1336.1339.1344.1346.1370.138.1384.1389.139.1390.1393.14.1414.142.1422.1427.1430.1437.144.1440.1456.1464.1468.1473.148.160.171.176.180.181.191.201.205.208.215.218.229.230.237.239.245.248.252.254.262.265.267.27.281.294.298.306.308.311.333.334.338.34.342.346.349.35.356.376.379.383.384.387.396.401.41.416.421.425.434.441.446.454.463.469.47.470.471.478.48.481.484.497.513.516.518.523.531.532.536.539.55.550.551.553.558.560.562.57.576.578.579.580.582.583.593.596.605.619.620.623.624.641.643.645.649.653.660.663.667.668.677.683.685.7.700.708.709.710.713.725.727.732.739.745.751.759.763.764.765.769.779.783.787.791.802.805.809.81.810.814.817.821.833.836.842.846.855.873.879.880.887.889.890.891.895.901.906.908.917.918.920.921.928.930.96.965.972.977.982.984.991.998.1.100.1001.1002.1005.1006.101.1010.1011.1012.1016.1017.1018.10

,path,type,filetype,num_id,grayscale_resize_path,resize_100_path
0,../data/train/Type_1/0.jpg,Type_1,jpg,0,../data/processed/grayscale_resize/train/0.jpg,../data/processed/resize_100/train/0.jpg
1,../data/train/Type_1/10.jpg,Type_1,jpg,10,../data/processed/grayscale_resize/train/10.jpg,../data/processed/resize_100/train/10.jpg
2,../data/train/Type_1/1013.jpg,Type_1,jpg,1013,../data/processed/grayscale_resize/train/1013.jpg,../data/processed/resize_100/train/1013.jpg
3,../data/train/Type_1/1014.jpg,Type_1,jpg,1014,../data/processed/grayscale_resize/train/1014.jpg,../data/processed/resize_100/train/1014.jpg
4,../data/train/Type_1/1019.jpg,Type_1,jpg,1019,../data/processed/grayscale_resize/train/1019.jpg,../data/processed/resize_100/train/1019.jpg


In [23]:
tdf100 = c.processing.transform_save_imgs(tdf, c.processing.resize_100); tdf100.head()

,path,num_id,grayscale_resize_path,resize_100_path
0,../data/test/0.jpg,0,../data/processed/grayscale_resize/test/0.jpg,../data/processed/resize_100/test/0.jpg
1,../data/test/1.jpg,1,../data/processed/grayscale_resize/test/1.jpg,../data/processed/resize_100/test/1.jpg
2,../data/test/10.jpg,10,../data/processed/grayscale_resize/test/10.jpg,../data/processed/resize_100/test/10.jpg
3,../data/test/100.jpg,100,../data/processed/grayscale_resize/test/100.jpg,../data/processed/resize_100/test/100.jpg
4,../data/test/101.jpg,101,../data/processed/grayscale_resize/test/101.jpg,../data/processed/resize_100/test/101.jpg


In [26]:
X100, y100 = c.processing.random_forest_transform(tdf100, 'resize_100_path', grayscale=False, test=True)
print(X100.shape)
print(y100.shape)

(512, 30000)
(512,)


In [25]:
X_trainrgb, X_testrgb, y_trainrgb, y_testrgb = train_test_split(
    Xrgb, yrgb, test_size=0.33, random_state=42)

NameError: name 'Xrgb' is not defined

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
score, ldf = cervix.submission.compute_losses('../data/dev_submission.csv')

In [ ]:
score

In [ ]:
ldf

In [27]:
c.data.check_image(df.resize_100_path[0])

AttributeError: 'DataFrame' object has no attribute 'resize_100_path'

In [ ]:
c.data.check_image(tdf.path[1])